In [1]:
import torch
import torch.nn as nn
import joblib
from transformers import DistilBertTokenizer, DistilBertModel


In [2]:
class F1BertModel(nn.Module):
    def __init__(self, num_classes):
        super(F1BertModel, self).__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        cls_output = bert_output.last_hidden_state[:, 0, :]
        x = self.dropout(cls_output)
        return self.fc(x)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Load label encoder
le = joblib.load("models/label_encoder.pkl")

# Create model
num_classes = len(le.classes_)
model = F1BertModel(num_classes=num_classes)

# Load trained weights
model.load_state_dict(torch.load("models/bert_model.pt", map_location=device))
model.to(device)
model.eval()

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [4]:
def predict_winner(text):
    # 1. Tokenize input
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt"
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    # 2. Predict
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        _, predicted = torch.max(outputs, dim=1)

    # 3. Convert number → driver name
    winner = le.inverse_transform(predicted.cpu().numpy())[0]

    return winner


In [ ]:
future_race_text = """e"""

print("Predicted next winner:", predict_winner(future_race_text))


Predicted next winner: Max Verstappen
